In [2]:
# Write your code here
import numpy as np
from types import SimpleNamespace

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [70]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))

print(X)

[[0.67583134 0.2143232 ]
 [0.30945203 0.7994661 ]
 [0.9958021  0.14223182]
 [0.07872553 0.18082381]
 [0.35964689 0.16961925]
 [0.58875932 0.61680751]
 [0.10538568 0.56573105]
 [0.00462964 0.4651192 ]
 [0.9756222  0.79942844]
 [0.59682237 0.32534966]
 [0.20634391 0.44272557]
 [0.2780414  0.87495784]
 [0.21315735 0.274245  ]
 [0.80718199 0.26836533]
 [0.26806287 0.07088178]
 [0.46720881 0.26420544]
 [0.88894204 0.28631831]
 [0.77376693 0.48724486]
 [0.46801905 0.96493021]
 [0.89822733 0.07903432]
 [0.24520427 0.18478708]
 [0.9054749  0.55383204]
 [0.37165898 0.83389703]
 [0.34877258 0.68165405]
 [0.22835057 0.02387229]
 [0.69611898 0.33685277]
 [0.3419926  0.27584087]
 [0.25134374 0.57010553]
 [0.33385622 0.42559779]
 [0.20192981 0.50515967]
 [0.58538723 0.42030016]
 [0.40344687 0.94394282]
 [0.04821238 0.32607379]
 [0.51893133 0.59845416]
 [0.04229511 0.24125679]
 [0.05425634 0.00773076]
 [0.32209779 0.40699871]
 [0.85917435 0.01347645]
 [0.71623561 0.4569535 ]
 [0.58907501 0.14639442]


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [34]:
#defining A, B, C and D
from scipy.optimize import minimize

# Define the objective function
def objective(x, y1, y2):
    x1, x2 = x
    y1, y2 = y
    return np.sqrt((x1 - y1)**2 + (x2 - y2)**2)

# Define the constraint functions
def constraint1(x):
    return x[0] - y1

def constraint2(x):
    return y1 - x[0]

def constraint3(x):
    return x[1] - y2

def constraint4(x):
    return y2 - x[1]

def constraint(x):
    

# Initial guess for x1 and x2
x0A = np.array([y1 + 1, y2 + 1])
x0B = np.array([y1 + 1, y2 - 1])
x0C = np.array([y1 - 1, y2 - 1])
x0D = np.array([y1 - 1, y2 + 1])


# Define the constraints in the form expected by the optimizer
constraintsA = [{'type': 'ineq', 'fun': constraint1},
               {'type': 'ineq', 'fun': constraint3}]
constraintsB = [{'type': 'ineq', 'fun': constraint1},
               {'type': 'ineq', 'fun': constraint4}]
constraintsC = [{'type': 'ineq', 'fun': constraint2},
               {'type': 'ineq', 'fun': constraint4}]
constraintsD = [{'type': 'ineq', 'fun': constraint2},
               {'type': 'ineq', 'fun': constraint3}]


# Perform the optimization
resultA = minimize(objective, x0A, args=(y1, y2), constraints=constraintsA)
resultB = minimize(objective, x0B, args=(y1, y2), constraints=constraintsB)
resultC = minimize(objective, x0C, args=(y1, y2), constraints=constraintsC)
resultD = minimize(objective, x0D, args=(y1, y2), constraints=constraintsD)

A = np.array(resultA.x)
B = np.array(resultB.x)
C = np.array(resultC.x)
D = np.array(resultD.x)

Optimal values: x1 = 0.20901926122951642, x2 = 0.37774758728699737
Minimum distance: 2.465720107482118e-15
[0.20901926 0.37774759]
0.20901926122951642
[0.20901913 0.37774759]


In [68]:
# write your answer here

# Define the objective function
def objective(x, y1, y2):
    x1, x2 = x
    y1, y2 = y
    return np.sqrt((x1 - y1)**2 + (x2 - y2)**2)

filtered_X_A = np.array([point for point in X if point[0] > y1 and point[1] > y2])
filtered_X_B = np.array([point for point in X if point[0] > y1 and point[1] < y2])
filtered_X_C = np.array([point for point in X if point[0] < y1 and point[1] < y2])
filtered_X_D = np.array([point for point in X if point[0] < y1 and point[1] > y2])

if len(filtered_X) == 0:
    raise ValueError("NaN")

# Find the point in filtered_X that minimizes the objective function
min_distance = float('inf')
optimal_point = None
for point in filtered_X_A:
    dist = objective(point, y1, y2)
    if dist < min_distance:
        min_distance = dist
        optimal_point_A = point

for point in filtered_X_B:
    dist = objective(point, y1, y2)
    if dist < min_distance:
        min_distance = dist
        optimal_point_B = point

for point in filtered_X_C:
    dist = objective(point, y1, y2)
    if dist < min_distance:
        min_distance = dist
        optimal_point_C = point

for point in filtered_X_D:
    dist = objective(point, y1, y2)
    if dist < min_distance:
        min_distance = dist
        optimal_point_D = point

# Assign the optimal values to vector A
A = np.array(optimal_point_A)
B = np.array(optimal_point_B)
C = np.array(optimal_point_C)
D = np.array(optimal_point_D)

print(A)
print(B)
print(C)
print(D)

0.07815236144144315
[0.26071605 0.43635845]
[0.21315735 0.274245  ]
[0.05425634 0.00773076]
[0.20634391 0.44272557]


**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [64]:
# write your answer here

r1ABC = ((B[1]-C[1])*(y[0]-C[0])+(C[0]-B[0])*(y[1]-C[1]))/((B[1]-C[1])*(A[0]-C[0])+(C[0]-B[0])*(A[1]-C[1]))
r2ABC = ((C[1]-A[1])*(y[0]-C[0])+(A[0]-C[0])*(y[1]-C[1]))/((B[1]-C[1])*(A[0]-C[0])+(C[0]-B[0])*(A[1]-C[1]))
r3ABC = 1-r1ABC-r2ABC

print(r1ABC,r2ABC,r3ABC)

r1CDA = ((D[1]-A[1])*(y[0]-A[0])+(A[0]-D[0])*(y[1]-A[1]))/((D[1]-A[1])*(C[0]-A[0])+(A[0]-D[0])*(C[1]-A[1]))
r2CDA = ((A[1]-C[1])*(y[0]-A[0])+(C[0]-A[0])*(y[1]-A[1]))/((D[1]-A[1])*(C[0]-A[0])+(A[0]-D[0])*(C[1]-A[1]))
r3CDA = 1-r1CDA-r2CDA

print(r1CDA, r2CDA, r3CDA)

1.341202508298138 -0.7686632568499804 0.42746074855184235
0.14280924403909337 0.40852605516600204 0.4486647007949046


Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [63]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [65]:
# write your answer here

r1CDA*f(C)+r2CDA*f(D)+r3CDA*f(A)

0.08842290988669628

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [ ]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [ ]:
# write your answer here